# UNIDADE 4: Otimização e desempenho

4.1 Otimização de hiperparâmetros e regularização

4.1.1 Técnicas de avaliação de modelos

4.1.1.1 Overfitting e underfitting

4.1.1.2 Interpretabilidade dos modelos

4.1.1.3 Questões éticas e de viés

4.1.2 Validação cruzada

4.1.3 GridSearchCV

4.1.4 Avaliação de métricas


# Instalação do Ambiente


Para executar essa Unidade crie um novo **Environments no Anaconda (Environments >> Create)** e  <span style="color:red">escolha a versão 11 para o Python</span>. Instale os pacotes necessários

In [ ]:
!pip install pandas

In [ ]:
!pip install numpy

In [ ]:
!pip install scikit-learn

# Avaliação do Desempenho

* **Erro de Treinamento vs. Erro de Validação/Teste**: Compare o erro do modelo no conjunto de treinamento com o erro nos conjuntos de validação e test.
* **Overfitting**: Se o <b>erro</b> no conjunto de <b>treinamento é muito baixo</b>, mas o erro no conjunto de <b>validação/teste é significativamente maior</b>, o modelo pode estar overfitting.
     * Muitas features confundem o modelo.
     * Usar um modelo complexo para um problema simples.
     * Pouca regularização.
</br></br>
* **Underfitting**: Se o <b>erro</b> no conjunto de <b>treinamento e nos conjuntos de validação/teste é alto</b>, o modelo pode estar underfitting.
     * Poucas features perdem detalhes importantes.
     * Usar um modelo simples para um problema complexo.
     * Excessiva regularização limita a flexibilidade do modelo.


# Métricas de avaliação: regressão

As métricas abaixo são identificadas também como **funções de perda**, o objetivo é encontrar um modelo que tenha a menor função de perda.

* **Mean Absolute Error** (MAE): Erro Absoluto Médio é a <b>média do valor absoluto dos erros (diferenças absolutas entre os valores preditos e os valores observados)</b>.  MAE é útil quando <b>se quer uma métrica que seja robusta a outliers</b> e deseja-se entender a média absoluta dos erros em termos simples.

* **Mean Squared Error** (MSE): Erro Médio Quadrático é a <b>média dos erros quadrados, pune erros maiores</b>. MSE é preferido quando se deseja <b>penalizar mais fortemente os erros maiores</b>.

* **Root Mean Square Error** (RMSE): Raiz do Erro Quadrático Médio é a <b>raiz quadrada da média dos erros quadrados</b>. RMSE é frequentemente usado em contextos onde é importante <b>manter as unidades do erro comparáveis com os dados originais e ao mesmo tempo penalizar erros maiores</b>. 

* **Coeficiente de Determinação** (R²): O R² <b>mede a proporção da variância nos valores dependentes que é explicada pelo modelo</b>.



### Sinais de Overfitting

* Treino: BAIXO erro (MAE, MSE, RMSE) e ALTO R²
* Teste: ALTO erro (MAE, MSE, RMSE) e BAIXO R²

### Sinais de Underfitting

* Treino: ALTO erro (MAE, MSE, RMSE) e BAIXO R²
* Teste: ALTO erro (MAE, MSE, RMSE) e BAIXO R²

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import pandas as pd
import numpy as np

import joblib

from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import r2_score


# Validação Cruzada (Cross-Validation)

Em muitos casos, a divisão simples em treino e teste é expandida com a técnica de validação cruzada. Validação Cruzada é uma técnica de avaliação que **divide o conjunto de dados em múltiplas partes** para validar o desempenho do modelo. Ela ajuda a  <span style="color:red">garantir que o modelo generaliza bem para dados não vistos</span>, ao invés de simplesmente memorizar o conjunto de treino. Nesta abordagem, o conjunto de dados é dividido em múltiplas partes, e o **modelo é treinado e testado várias vezes** em diferentes subdivisões dos dados. 


### Características

* Avalia a  <span style="color:red">capacidade de generalização do modelo</span> em diferentes subconjuntos dos dados.
* Usada para avaliar a performance do modelo e  <span style="color:red">garantir que ele não está overfitado ou underfitado</span>.
    * Overfitting se torna evidente se há uma grande variação nos erros entre os subconjuntos.
* Concentra-se em  <span style="color:red">testar o modelo em diferentes divisões dos dados</span> para assegurar generalização.
* Usada durante a  <span style="color:red">fase de avaliação do modelo</span>, para medir sua performance e generalização.

### Métodos de Validação Cruzada:

* **k-Fold Cross Validation**: Divide os dados em k dobras; em cada iteração, uma dobra é usada para teste e as outras k-1 para treino.
* **Leave-One-Out Cross Validation (LOOCV)**: Cada instância é usada uma vez como conjunto de teste enquanto o restante serve como treino.
* **Stratified k-Fold**: Similar ao k-Fold, mas preserva a proporção de classes em cada dobra (útil para dados desbalanceados).
* **Time Series Cross Validation**: Para séries temporais, respeita a sequência temporal ao dividir os dados.

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">
<h3>k-Fold Cross Validation</h3>
Realiza uma validação cruzada k-fold, onde os dados são divididos em k dobras (subconjuntos) e o modelo é treinado em k-1 dobras enquanto é testado na dobra restante. Este processo é repetido k vezes, cada vez com uma diferente dobra sendo usada como teste.

<span style="color:red">A validação cruzada deve ser feita durante o processo de treinamento e avaliação do modelo</span> para obter uma estimativa confiável da performance do modelo em dados não vistos. 

<b>Objetivo da Validação Cruzada</b>
* Avaliar a Generalização:
    * Estimar como o modelo performa em dados não vistos (dados de teste).
    * Detectar problemas de overfitting e underfitting.
* Seleção de Modelo e Hiperparâmetros:
    * <b>Comparar diferentes modelos e configurações de hiperparâmetros</b>.
    * Escolher a melhor configuração com base em métricas de desempenho.

</div>

In [2]:
 model = joblib.load('../models/model.pkl')

C:\Users\joelma\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.1 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
df_normalizado = pd.read_csv("../data/processed/predicao_cargas_trabalho_normalizado.csv")

X = df_normalizado.loc[:, df_normalizado.columns != 'casos_pendentes']
y = df_normalizado["casos_pendentes"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [4]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-scores)

print(f'RMSE Score: {np.mean(rmse_scores)}')

RMSE Score: 0.07132818876514097


In [5]:
scores = cross_val_score(model, X, y, cv=5, scoring='r2')
print(f'Mean R² Score: {scores.mean()}')

Mean R² Score: 0.7934604854754571


# Otimização de Hiperparâmetros

Otimização de Hiperparâmetros é o processo de encontrar a **melhor combinação de hiperparâmetros** para um modelo de aprendizado de máquina. <span style="color:red">Hiperparâmetros</span> são parâmetros que não são aprendidos durante o treinamento do modelo, mas são <span style="color:red">definidos antes do treinamento</span> e influenciam o comportamento do algoritmo.

### Características

* Encontra os <span style="color:red">melhores valores de hiperparâmetros</span> para otimizar o desempenho do modelo.
* Utilizada para <span style="color:red">melhorar o modelo</span> ajustando parâmetros não aprendidos.
* Concentra-se nos <span style="color:red">parâmetros externos</span> ao treinamento do modelo.
* Usada durante a <span style="color:red">fase de construção e ajuste do modelo</span>.

### Métodos de Otimização:

* **Grid Search**: Explora exaustivamente uma grade de combinações de hiperparâmetros.
* **Random Search**: Seleciona combinações aleatórias de hiperparâmetros.
* **Bayesian Optimization**: Utiliza métodos probabilísticos para explorar o espaço de hiperparâmetros de forma mais eficiente.
* **Genetic Algorithms**: Utiliza princípios de evolução natural para encontrar a melhor combinação de hiperparâmetros.

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">
  <h3>Grid Search</h3>
Testa <span style="color:red">todas as combinações possíveis dos hiperparâmetros fornecidos pelo usuário</span>, realizando uma busca exaustiva sobre o espaço de hiperparâmetros especificado. Usa validação cruzada para avaliar a performance do modelo para cada combinação de hiperparâmetros, dividindo os dados em múltiplas dobras (folds) e treinando o modelo em diferentes subconjuntos dos dados.

</div>

<h2>✅ ElasticNetCV</h2>

<div style="background-color: #ffcccc; padding: 20px; border-radius: 10px;">
ElasticNetCV é uma implementação da regressão Elastic Net que incorpora <b>validação cruzada (cross-validation) para selecionar os melhores valores de hiperparâmetros</b>. O Elastic Net é uma técnica de regularização que combina os métodos L1 (Lasso) e L2 (Ridge). </div>

In [6]:
pipeline = Pipeline([
    ('model', ElasticNetCV())
])

# Definir a grade de parâmetros
param_grid = {
    'model__fit_intercept':[True, False],
    'model__l1_ratio':[1, 0.1, 0.001, 0.005],
    'model__tol': [.1, .5, .7],
    'model__eps': [0.001, 0.005, 0.01],
    'model__alphas': [[4, 1, 0.1, 0.001, 0.0005, 5e-4],[0.001, 0.0005],[0.1, 0.001, 0.0010]],
    'model__selection': ['cyclic', 'random']
}

grid_search = GridSearchCV(pipeline, param_grid, 
                           cv=5, 
                           scoring='neg_mean_squared_error', 
                           return_train_score=True,
                           verbose=2, 
                           n_jobs=-1)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits


GridSearchCV(cv=5, estimator=Pipeline(steps=[('model', ElasticNetCV())]),
             n_jobs=-1,
             param_grid={'model__alphas': [[4, 1, 0.1, 0.001, 0.0005, 0.0005],
                                           [0.001, 0.0005],
                                           [0.1, 0.001, 0.001]],
                         'model__eps': [0.001, 0.005, 0.01],
                         'model__fit_intercept': [True, False],
                         'model__l1_ratio': [1, 0.1, 0.001, 0.005],
                         'model__selection': ['cyclic', 'random'],
                         'model__tol': [0.1, 0.5, 0.7]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=2)

In [7]:
# Imprimir os melhores parâmetros
print("Melhores parâmetros: ", grid_search.best_params_)

Melhores parâmetros:  {'model__alphas': [0.001, 0.0005], 'model__eps': 0.01, 'model__fit_intercept': True, 'model__l1_ratio': 1, 'model__selection': 'random', 'model__tol': 0.5}


In [8]:
# Ajustar o modelo com os melhores parâmetros
modelElasticNetCV = grid_search.best_estimator_
modelElasticNetCV.fit(X_train, y_train)

predictions = modelElasticNetCV.predict(X_test)

print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
print('R2:', r2_score(y_test, predictions))

MAE: 0.05535473040319945
MSE: 0.005242314521086722
RMSE: 0.07240382946423982
R2: 0.7922575293144752


In [9]:
models_result = []

metric_model = {
    'MODEL': 'ElasticNetCV+GridSearchCV',
    'MAE': metrics.mean_absolute_error(y_test, predictions),
    'MSE': metrics.mean_squared_error(y_test, predictions),
    'RMSE': np.sqrt(metrics.mean_squared_error(y_test, predictions)),
    'R2': r2_score(y_test, predictions)
}

models_result.append(metric_model)

<h2>✅ Ridge</h2>

<div style="background-color: #ffcccc; padding: 20px; border-radius: 10px;">
A regressão ridge é uma técnica de regularização usada para aumentar a robustez da regressão linear, especialmente quando há <b>multicolinearidade entre as variáveis preditoras. A regressão ridge adiciona uma penalização do tipo L2 ao termo de erro da regressão linear, que é o quadrado da magnitude dos coeficientes.</b> </div>

In [10]:
pipeline = Pipeline([
    ('model', Ridge())
])


# Definir a grade de parâmetros
param_grid = {
    'model__fit_intercept':[True, False],
    'model__alpha':[1, 0.1, 0.001, 0.005],
    'model__tol': [0.001, 0.005, 0.01]
}

grid_search = GridSearchCV(pipeline, param_grid, 
                           cv=5, 
                           scoring='neg_mean_squared_error', 
                           return_train_score=True,
                           verbose=2, 
                           n_jobs=-1)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(cv=5, estimator=Pipeline(steps=[('model', Ridge())]), n_jobs=-1,
             param_grid={'model__alpha': [1, 0.1, 0.001, 0.005],
                         'model__fit_intercept': [True, False],
                         'model__tol': [0.001, 0.005, 0.01]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=2)

In [11]:
# Imprimir os melhores parâmetros
print("Melhores parâmetros: ", grid_search.best_params_)

Melhores parâmetros:  {'model__alpha': 0.1, 'model__fit_intercept': False, 'model__tol': 0.001}


In [12]:
# Ajustar o modelo com os melhores parâmetros
modelRidge = grid_search.best_estimator_
modelRidge.fit(X_train, y_train)

predictions = modelRidge.predict(X_test)

print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
print('R2:', r2_score(y_test, predictions))

MAE: 0.05509190723806521
MSE: 0.005210744006268944
RMSE: 0.07218548334858571
R2: 0.7935086058614237


In [13]:
metric_model = {
    'MODEL': 'Ridge+GridSearchCV',
    'MAE': metrics.mean_absolute_error(y_test, predictions),
    'MSE': metrics.mean_squared_error(y_test, predictions),
    'RMSE': np.sqrt(metrics.mean_squared_error(y_test, predictions)),
    'R2': r2_score(y_test, predictions)
}

models_result.append(metric_model)

In [14]:
df = pd.DataFrame(models_result)
df = df.sort_values(by='R2', ascending=False).reset_index(drop=True)
df

,MODEL,MAE,MSE,RMSE,R2
0,Ridge+GridSearchCV,0.055092,0.005211,0.072185,0.793509
1,ElasticNetCV+GridSearchCV,0.055355,0.005242,0.072404,0.792258


In [20]:
df_result = pd.read_csv("result.csv")
result = pd.concat([df_result, df], axis=0, ignore_index=True).sort_values(by='R2', ascending=False).reset_index(drop=True)
result.to_csv('./result_valid.csv', index=False)
result

,MODEL,MAE,MSE,RMSE,R2
0,LinearRegression,0.054777,0.004913,0.070089,0.803984
1,BayesianRidge,0.054784,0.004913,0.070095,0.803955
2,Ridge,0.054840,0.004920,0.070141,0.803694
3,RandomForestRegressor,0.055808,0.005032,0.070939,0.799203
4,GradientBoostingRegressor,0.056144,0.005112,0.071496,0.796039
5,Ridge+GridSearchCV,0.055092,0.005211,0.072185,0.793509
6,ElasticNetCV+GridSearchCV,0.055355,0.005242,0.072404,0.792258
7,DecisionTreeRegressor,0.057536,0.005374,0.073310,0.785558
8,MLPRegressor,0.059035,0.005706,0.075538,0.772321
9,SVR,0.063235,0.006357,0.079729,0.746360


<div style="background-color: #ff87b2; padding: 20px; border-radius: 10px;">
<h3>Atividade 4:</h3>

Para o estudo de caso do Predição de Cargas de Trabalho para Juízes cujo objetivo prever o volume de trabalho em diferentes tribunais com base em fatores como tamanho da população, tipos de casos comuns na região, etc.

Execute a otimização de hiperparâmetros para o : BayesianRidge, RandomForestRegressor, GradientBoostingRegressor, e MLPRegressor.

Monte um DataFrame comparativo mostrando as métricas MAE, MSE, RMSE, R2 de cada modelo.

Poste no AVA o Jupyter Notebook ou o link para o repositório GitHub.
  
</div>.

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">
<h3> Comparação de Modelos</h3>
  Comparação de diferentes modelos usando validação cruzada.

O ideal é que a comparação de modelos usando validação cruzada seja feita em dados que não foram vistos pelo modelo durante o treinamento. Para garantir que estamos avaliando a capacidade de generalização dos modelos, é <span style="color:red">importante usar uma abordagem que separe claramente os dados de treinamento, validação e teste</span>.
     
</div>

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

# Definir os modelos a serem comparados
models = {
    'ElasticNetCV': ElasticNetCV(alphas=[4, 1, 0.1, 0.001, 0.0005, 0.0005], eps=0.005, fit_intercept=True, l1_ratio=1, selection='random', tol=0.7),
    'Ridge': Ridge(alpha=1, fit_intercept=False, tol=0.001)
}

# Avaliar cada modelo usando cross_val_score
for name, model in models.items():
    pipeline = Pipeline([
        ('model', model)
    ])
    scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='r2', n_jobs=-1)
    print(f'{name} - R²: {scores.mean()}, Std: {scores.std()}')

ElasticNetCV - R²: 0.7955821770292666, Std: 0.011187861166287014
Ridge - R²: 0.7946432812406021, Std: 0.010920952959887454


<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">
* <b>Dados de Treinamento e Validação</b>: A validação cruzada e a otimização de hiperparâmetros são feitas nos dados de treinamento.</br>
* <b>Avaliação de Modelos</b>: A comparação de modelos é realizada no conjunto de validação, que não foi visto durante o treinamento.</br>
* <b>Avaliação Final</b>: A performance final do modelo é avaliada no conjunto de teste, que também não foi visto durante o treinamento e a validação. </br>  
</div>